<a href="https://colab.research.google.com/github/laynz28/AI-Song-Cover-RVC/blob/main/TrainingV2_NoUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Mount Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Unzip File From Drive

!unzip -n /content/drive/MyDrive/assets/KiwKiw.zip -d /content
%cd /content/KiwKiw
!pip install -r requirements.txt -q

In [ ]:
import os

# @title Create Model Folder
model_name = "kafka" # @param {type:"string"}
exp_dir = model_name
dataset = "/content/drive/MyDrive/dataset/kafka" # @param {type:"string"}
sample_rate = "48000" # @param ["40000", "48000"]
ksample_rate = "48k"
if sample_rate == "40000":
  ksample_rate = "40k"
else:
  ksample_rate = "48k"
version = "v2" # @param ["v1", "v2"]
version19 = version

f0method = "rmvpe_gpu" # @param ["pm", "dio", "harvest", "rmvpe", "rmvpe_gpu"]

save_frequency = 50 # @param {type:"slider", min:0, max:50, step:1}
epoch = 10 # @param {type:"integer"}
batch_size = "7" # @param {type:"string"}
cache_gpu = True # @param {type:"boolean"}

now_dir = "/content/KiwKiw"

os.makedirs("%s/logs/%s" % (now_dir, exp_dir), exist_ok=True)
f = open("%s/logs/%s/preprocess.log" % (now_dir, exp_dir), "w")
os.makedirs("%s/logs/%s" % (now_dir, exp_dir), exist_ok=True)
f = open("%s/logs/%s/extract_f0_feature.log" % (now_dir, exp_dir), "w")
f.close()

In [ ]:
# @title Process Data
command = f"python infer/modules/train/preprocess.py '{dataset}' {sample_rate} 2 '{now_dir}/logs/{exp_dir}' False 3.0"
print(command)
!{command}

In [ ]:
# @title Feature Extraction

if f0method != "rmvpe_gpu":
  command = f"python infer/modules/train/extract/extract_f0_print.py '{now_dir}/logs/{exp_dir}' 2 '{f0method}'"
else:
  command = f"python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 '{now_dir}/logs/{exp_dir}' True"
print(command)
!{command}

command = f"python infer/modules/train/extract_feature_print.py cuda:0 1 0 0 '{now_dir}/logs/{exp_dir}' '{version}'"
print(command)
!{command}

In [ ]:
# @title Train Feature Index
import numpy as np
import faiss


def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "请先进行特征提取!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "请先进行特征提取！"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * config.n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    # index = faiss.index_factory(256if version19=="v1"else 768, "IVF%s,PQ128x4fs,RFlat"%n_ivf)
    infos.append("training")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("adding")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "成功构建索引，added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

result_generator = train_index(exp_dir, version)

for result in result_generator:
    print(result)

In [ ]:
# @title Train Model

from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT

# Remove the logging setup

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    # Replace logger.debug, logger.info with print statements
    print("Write filelist done")
    print("Use gpus:", str(gpus16))
    if pretrained_G14 == "":
        print("No pretrained Generator")
    if pretrained_D15 == "":
        print("No pretrained Discriminator")
    if version19 == "v1" or sr2 == "40k":
        config_path = "configs/v1/%s.json" % sr2
    else:
        config_path = "configs/v2/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    # Use PIPE to capture the output and error streams
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    # Print the command's output as it runs
    for line in p.stdout:
        print(line.strip())

    # Wait for the process to finish
    p.wait()
    return "训练结束, 您可查看控制台训练日志或实验文件夹下的train.log"

if version == 'v1':
    if ksample_rate == '40k':
        G_path = 'assets/pretrained/f0G40k.pth'
        D_path = 'assets/pretrained/f0D40k.pth'
    elif ksample_rate == '48k':
        G_path = 'assets/pretrained/f0G48k.pth'
        D_path = 'assets/pretrained/f0D48k.pth'
elif version == 'v2':
    if ksample_rate == '40k':
        G_path = 'assets/pretrained_v2/f0G40k.pth'
        D_path = 'assets/pretrained_v2/f0D40k.pth'
    elif ksample_rate == '48k':
        G_path = 'assets/pretrained_v2/f0G48k.pth'
        D_path = 'assets/pretrained_v2/f0D48k.pth'

result_generator = click_train(
    exp_dir,
    ksample_rate,
    True,
    0,
    save_frequency,
    epoch,
    batch_size,
    True,
    G_path,
    D_path,
    0,
    cache_gpu,
    False,
    version,
)
print(result_generator)

In [ ]:
#@title Copy Training Result to Drive
!mkdir -p /content/drive/MyDrive/Model/{exp_dir}

!cp /content/KiwKiw/logs/{exp_dir}/added_*.index /content/drive/MyDrive/Model/{exp_dir}
!cp /content/KiwKiw/logs/{exp_dir}/total_*.npy /content/drive/MyDrive/Model/{exp_dir}
!cp /content/KiwKiw/assets/weights/{exp_dir}.pth /content/drive/MyDrive/Model/{exp_dir}

%cd /content/drive/MyDrive/Model/{exp_dir}
!zip {exp_dir}.zip added_*.index total_*.npy {exp_dir}.pth
%cd /content/KiwKiw

In [ ]:
#@title Export Finished Model to HuggingFace
#Credit to LollenApe for this cell!

#TODO:
#maybe remove glob dependency?

import os
import zipfile
import glob

assert experiment_name, "No experiment name found."

#@markdown Use this option if you wish to manually use a specific epoch value. *Leave this blank if your model finished training.*
manual_epoch_number = "" #@param {type:"string"}
num_epochs = int(manual_epoch_number) if manual_epoch_number.isdigit() else None

# Construct the weights path based on the provided number of epochs
if num_epochs is not None:
  weights_path = f"/content/KiwKiw/weights/{experiment_name}_e{num_epochs}*"
else:
  print("Autodetecting epoch count...")
  potential = f"/content/KiwKiw/weights/{experiment_name}.pth"
  if os.path.exists(potential):
    weights_path = f"/content/KiwKiw/weights/{experiment_name}"
  else:
    currentMax = 0
    for r, _, f in os.walk("/content/KiwKiw/weights/"):
      for name in f:
        if(name.endswith(".pth") and (name!=experiment_name+".pth")):
          if(name.find(experiment_name)==-1):
            continue
          pot = name.split('_')
          ep=pot[len(pot)-2][1:]
          if(not ep.isdecimal()):
            continue
          ep=int(ep)
          if ep>currentMax:
            currentMax=ep
    num_epochs=currentMax
    weights_path = f"/content/KiwKiw/weights/{experiment_name}_e{num_epochs}*"
  print(f"Model with path {weights_path} found automatically.")

weights_files = glob.glob(weights_path + ".pth")
logs_path = f"/content/KiwKiw/logs/{experiment_name}/added_*_1_{experiment_name}_{model_architecture}.index"

print(f"Searching for weights files in: {weights_path}.pth")
print(f"Searching for logs file in: {logs_path}")

if weights_files and any(glob_result := glob.glob(logs_path)):
    log_file = glob_result[0]
    output_folder = "/content/toHF"
    os.makedirs(output_folder, exist_ok=True)
    output_zip_path = f"{output_folder}/{experiment_name}.zip"
    with zipfile.ZipFile(output_zip_path, 'w') as zipf:
        for weights_file in weights_files:
            zipf.write(weights_file, os.path.basename(weights_file))
        zipf.write(log_file, os.path.basename(log_file))
    print(f"The files have been added to the zip folder: {output_zip_path}")
else:
    print(f"Found weights files: {weights_files}")
    print(f"Found logs files: {glob_result}")
    raise Exception("Couldn't find your model files. Check the found file results above. (Did you run Index Training?)")

from huggingface_hub import login, HfApi
#@markdown Enter HuggingFace Token (set Role to 'write'):
hftoken = "Obtain this at https://huggingface.co/settings/tokens" #@param {type:"string"}

# Login
login(token=f"{hftoken}")

#@markdown Set this option to true if you wish to create a new repository for this model.
create_new_repo = False #@param {type:"boolean"}
#@markdown Enter the HuggingFace repository name you wish to send the model to:
repoid = "username/RepoName" #@param {type:"string"}

if create_new_repo:
    from huggingface_hub import create_repo
    create_repo(repoid)
    print(f"Repo '{repoid}' created successfully!")

api = HfApi()
# Upload folders
api.upload_folder(folder_path="/content/toHF",
                 repo_id=f"{repoid}",
                 repo_type="model")